In [1]:
import time
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from yandex import Translater
from pandas import Series
from collections import Counter
from google.cloud import translate
import types
# client=translate.Client()

In [3]:
df = pd.read_csv('../data/comments.csv')
df.fillna('', inplace=True)

In [4]:
df.shape

(316043, 3)

In [5]:
df.head()

,channelId,community,text
0,UC5tp0GG3-cvlnflj5NuA9xQ,9,"i think it's the tree stump, could be a a slee..."
1,UC5tp0GG3-cvlnflj5NuA9xQ,9,Lothrazar. use the dream rod on the tree stump...
2,UC5tp0GG3-cvlnflj5NuA9xQ,9,whoops sorry posted that half way through the ...
3,UC5tp0GG3-cvlnflj5NuA9xQ,9,lol kinda pointless to say so now but by the l...
4,UC5tp0GG3-cvlnflj5NuA9xQ,9,"Thanks, and it wasn't a spoiler, I usualy have..."


In [20]:
i=0

fieldnames=['channelId','community','text','language']
out=open('../data/comments_lan.csv', 'a')
writer = csv.DictWriter(out, fieldnames=fieldnames)
# writer.writeheader()  
    
for text in df.text:
    if i>=52463:
        if len(text)>35:
            txt=text[0:35]
        else:
            txt=text

        lan=client.detect_language(txt)
    #     conf=conf+lan['confidence']
        df.loc[i, 'language'] = lan['language']

        writer.writerow(df.iloc[i].to_dict())
        
        if i%100==0:
            time.sleep(1)
    
    i+=1
        
out.close()

In [31]:
# l=0
# j=0
# for i in range (0, len(df)):
#     if len(df.text[i])>35:
#         text=df.text[i][0:35]
#         j+=1
#     else:
#         text=df.text[i]
        
#     l+=len(text)


In [32]:
# l=0
# for text in df.text:
#     if len(text)>35:
#         txt=text[0:35]
#     else:
#         txt=text
    
#     l+=len(txt)

In [33]:
# l=9410119

## Length of csv comments by language file

In [50]:
with open ('../data/comments_translated2.csv', 'r') as csvfile:
    fileObj=csv.reader(csvfile)
    num = sum(1 for row in fileObj)
    
num

125849

In [12]:
df2 = pd.read_csv('../data/comments_lan.csv')
df2.fillna('', inplace=True)

In [38]:
counter=Counter(Series(df2.language)).most_common(100)
# print (counter)
freq_lan= [counter [i][0] for i in range (0,len(counter))]

## Yandex detection algorithm

In [9]:
tr=Translater()
tr.set_key(os.environ.get('YANDEX_API'))

fieldnames=['channelId','community','text','language']
out=open('../data/comments_lan2.csv', 'a')
writer = csv.DictWriter(out, fieldnames=fieldnames)
# writer.writeheader()  

In [80]:
i=0
for lan in df2.language:
    if i>=244969:
        if (lan not in freq_lan) and (lan!='und'):
            tr.set_text(df2.iloc[i].text)
            df2.loc[i, 'language'] = tr.detect_lang()
            writer.writerow(df2.iloc[i].to_dict())
    #         print(df2.iloc[i].to_dict())
        else:
            writer.writerow(df2.iloc[i].to_dict())
    #         print(df2.iloc[i].to_dict())
    i+=1
    
out.close()

In [4]:
df3 = pd.read_csv('../data/comments_lan2.csv')
df3.fillna('', inplace=True)

In [5]:
i=0
st='Failed to detect the language! (response code Bad Request'

for lan in df3.language:
    if (lan ==st):
        df3.loc[i,'language']='und'
    i+=1

In [6]:
counter=Counter(Series(df3.language)).most_common(10)
freq_lan= [counter [i][0] for i in range (0,len(counter))]
print(freq_lan)
print(counter)

['en', 'de', 'ru', 'es', 'pt', 'und', 'it', 'fr', 'nl', 'tr']
[('en', 190408), ('de', 39244), ('ru', 34905), ('es', 26823), ('pt', 10944), ('und', 7834), ('it', 2455), ('fr', 2358), ('nl', 162), ('tr', 119)]


In [7]:
df3.shape

(316043, 4)

In [8]:
df3.head()

,channelId,community,text,language
0,UC5tp0GG3-cvlnflj5NuA9xQ,9,"i think it's the tree stump, could be a a slee...",en
1,UC5tp0GG3-cvlnflj5NuA9xQ,9,Lothrazar. use the dream rod on the tree stump...,en
2,UC5tp0GG3-cvlnflj5NuA9xQ,9,whoops sorry posted that half way through the ...,en
3,UC5tp0GG3-cvlnflj5NuA9xQ,9,lol kinda pointless to say so now but by the l...,en
4,UC5tp0GG3-cvlnflj5NuA9xQ,9,"Thanks, and it wasn't a spoiler, I usualy have...",en


## YANDEX translation of text detected in other languages

In [34]:
tr=Translater()
tr.set_key(os.environ.get('YANDEX_API3'))
tr.set_to_lang('en')

fieldnames=['channelId','community','text','language','translation']
out2=open('../data/comments_translated2.csv', 'a')
writer2 = csv.DictWriter(out2, fieldnames=fieldnames)
# writer2.writeheader()  

In [36]:
i=0
foreign=0
    
for lan in df3.language:
    if i>=309850:
        if (lan in freq_lan) and (lan!='und'):
            txt=df3.iloc[i].text
            
            if lan=='en':
                df3.loc[i, 'translation'] =txt
                writer2.writerow(df3.iloc[i].to_dict())
            else:
                ## Yandex Translation
                tr.set_text(txt)
                tr.set_from_lang(lan)
                
                df3.loc[i, 'translation'] = tr.translate()
                writer2.writerow(df3.iloc[i].to_dict())
                foreign+=len(txt)
            
            if (i%5==0):
                time.sleep(1.5)
                
            if foreign>=900000:
                break
                print(i)
    i+=1
        
out2.close()

In [35]:
tr.set_key(os.environ.get('YANDEX_API3'))
tr.set_text('hola mundo')
tr.set_to_lang('en')
tr.set_from_lang('es')
tr.translate()

'hello world'

## Find line last comment

In [2]:
df5 = pd.read_csv('../data/comments_translated2.csv')
df5.fillna('', inplace=True)
l=len(df5)
print (l)
com= df5.iloc[l-1].text
print(df5.iloc[l-1])

307450
channelId                               UCrd7UM5OBtB0n1zl41qwCUw
community                                                     87
text           Юджин от меча разум понижается(ставьте лайки ч...
language                                                      ru
translation    Eugene from the sword of the mind is reduced(l...
Name: 307449, dtype: object


In [9]:
i=0

for lan in df3.language:
    
    if lan=='ru':
        if df3.iloc[i].text== com:
            print(i)
#             print(df3.iloc[i])
            break
        
    i+=1

316042


In [33]:
print(df5.iloc[l-1])
print (df3.iloc[i])
print (df3.iloc[i+1])
print(i+1)

channelId                               UCrd7UM5OBtB0n1zl41qwCUw
community                                                     87
text           рисуем арты для юджина по сампу (без них не бу...
language                                                      ru
translation    draw the arts to Eugene for Sampo (without the...
Name: 301862, dtype: object
channelId                               UCrd7UM5OBtB0n1zl41qwCUw
community                                                     87
text           рисуем арты для юджина по сампу (без них не бу...
language                                                      ru
translation    draw the arts to Eugene for Sampo (without the...
Name: 309849, dtype: object
channelId                               UCrd7UM5OBtB0n1zl41qwCUw
community                                                     87
text           Юджин как всегда монтаж на высоте,сюжет 500/10...
language                                                      ru
translation    Eugene, as always, 

In [27]:
len(df5)

301863

## Translate with Google

In [98]:
fieldnames=['channelId','community','text','language','translation']
out2=open('../data/comments_translated2.csv', 'a')
writer2 = csv.DictWriter(out2, fieldnames=fieldnames)

In [99]:
i=0
foreign=0

for lan in df3.language:
    if i>=166055:
        if (lan in freq_lan) and (lan!='und'):
            txt= df3.iloc[i].text
            
            if lan=='en':
                df3.loc[i, 'translation'] =txt
                writer2.writerow(df3.iloc[i].to_dict())
            else:
                tr=client.translate(txt, target_language='en',source_language=lan)
                df3.loc[i, 'translation']=tr['translatedText']
                writer2.writerow(df3.iloc[i].to_dict())
                foreign+=len(txt)
                
            if (i%100==0):
                time.sleep(1)
                
            if foreign>=4000000:
                break
                print(i)
                
    i+=1
        
out2.close()

In [54]:
df3.head()

,channelId,community,text,language
0,UC5tp0GG3-cvlnflj5NuA9xQ,9,"i think it's the tree stump, could be a a slee...",en
1,UC5tp0GG3-cvlnflj5NuA9xQ,9,Lothrazar. use the dream rod on the tree stump...,en
2,UC5tp0GG3-cvlnflj5NuA9xQ,9,whoops sorry posted that half way through the ...,en
3,UC5tp0GG3-cvlnflj5NuA9xQ,9,lol kinda pointless to say so now but by the l...,en
4,UC5tp0GG3-cvlnflj5NuA9xQ,9,"Thanks, and it wasn't a spoiler, I usualy have...",en


In [100]:
foreign

4000103

## Counting languages in translated file

In [30]:
count=Counter(df5.language).most_common

In [31]:
count

<bound method Counter.most_common of Counter({'en': 105215, 'de': 9163, 'pt': 8332, 'it': 1335, 'ru': 667, 'fr': 516, 'es': 478, 'nl': 84, 'tr': 58})>

In [105]:
i=0
eng=0
foreign=0

for lan in df3.language:
    if i>=136905:
        if (lan in freq_lan) and (lan!='und'):       
            if lan=='en':
                eng+=1
            else:
                foreign+=len(df3.iloc[i].text)
                
            if foreign>=4000000:
                break
                print(i)
                
#             if i==166054:
#                 break
    i+=1

In [42]:
i

168880

In [46]:
df3.shape


(316043, 5)

In [48]:
df3.loc[1,'translation']=''

In [56]:
tr=client.translate('Hola',target_language='en',source_language='es')

In [57]:
tr

{'input': 'Hola', 'translatedText': 'Hello'}

In [59]:
tr['translatedText']

'Hello'

In [61]:
i

136905

In [28]:
df5.to_csv('../data/comments_translated2.csv', index=False)